In [ ]:
# This code is used to scrap the data from the mls salaries

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time


# Webscrapping the MLS Players Association Salary Guide
url = "https://mlsplayers.org/resources/salary-guide"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the table with class "salary-report" and id "salary-report"
    table = soup.find("table", {"class": "salary-report", "id": "salary-report"})

    # Initialize lists to hold the data
    first_names = []
    last_names = []
    teams = []
    positions = []
    base_salaries = []
    guaranteed_compensations = []

    if table:
        # Loop through the table rows and extract data
        for row in table.find_all("tr"):
            columns = row.find_all("td")
            if len(columns) >= 2:
                first_names.append(columns[0].get_text())
                last_names.append(columns[1].get_text())
                teams.append(columns[2].get_text())
                positions.append(columns[3].get_text())
                base_salaries.append(columns[4].get_text())
                guaranteed_compensations.append(columns[5].get_text())

        # Create a DataFrame from the lists
        data = pd.DataFrame(
            {
                "first_name": first_names,
                "last_name": last_names,
                "team": teams,
                "position": positions,
                "base_salary": base_salaries,
                "guaranteed_compensation": guaranteed_compensations,
            }
        )
    else:
        print("Table not found on the page.")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

# adding day of collection

data["colletion_date"] = time.strftime("%Y-%m-%d")

# Removing comma and dollar sign from the base salary and guaranteed compensation columns, and converting them to numeric values

data["base_salary"] = data["base_salary"].str.replace(",", "").str.replace("$", "")
data["guaranteed_compensation"] = (
    data["guaranteed_compensation"].str.replace(",", "").str.replace("$", "")
)

data["base_salary"] = pd.to_numeric(data["base_salary"])
data["guaranteed_compensation"] = pd.to_numeric(data["guaranteed_compensation"])

# use slite to transform the data into a database


In [ ]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("salaries_app").getOrCreate()

# Assuming data_scrapped is your Pandas DataFrame
spark_df = spark.createDataFrame(data)

# Define variables for table creation
table_name = "salaries"
checkpoint_path = "/tmp/your_checkpoint_path"

# Write the DataFrame to a Delta table
(spark_df.write
  .format("delta")
  .mode("overwrite")  # You can change the mode as needed
  .saveAsTable(table_name))
